In [ ]:
# libraries
# ----------
import pandas as pd
import os
#import os.path
from os import path
import glob
import pyodbc
import pysftp
from io import BytesIO

import numpy as np
import requests, zipfile, io
from datetime import datetime, timedelta
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)
from datetime import date, timedelta
import time
import warnings
warnings.filterwarnings("ignore")

BOX_PATH = '/Users/raksha/Box Sync/Mondelez: Demand forecasts during COVID-19/4. EDA & Descriptive analytics'
LOCAL_PATH = '/Users/raksha/Box Sync/Accounts/Mondelez'

In [ ]:
# UPDATE with your Mondelez LAN ID & Password to allow the script to connect to Hive and download the raw POS dataset
mdlz_lan_id = "bwn2456"
mdlz_lan_pwd = ""

In [ ]:
# Connection to Hive to read in POS raw data
CONNECTION_STRING = ';'.join(f"""
Description=Hortonworks Knox DSN
Driver=/opt/cloudera/hiveodbc/lib/universal/libclouderahiveodbc.dylib
Host=mdzusvpclhdp101.mdzprod.local
port=8443
HttpPath=gateway/mondelez/hive
schema=default
ServiceDiscoveryMode=0
HiveServerType=2
AuthMech=3
ThriftTransport=2
SSL=1
TwoWaySSL=0
AllowSelfSignedServerCert=1
uid={mdlz_lan_id}
pwd={mdlz_lan_pwd}
""".splitlines())

In [ ]:
# Connection to edge node to write out POS model output data
edge_node = pysftp.Connection(host="10.54.252.11", username=mdlz_lan_id, password=mdlz_lan_pwd)

# Table of Contents

* [0. Raw data Connections](#first-bullet)
* [1. Read in POS Data](#second-bullet)
* [2. Read in State Projections](#third-bullet)
* [3. Pre-Processing of Modeling Data](#fourth-bullet)
* [4. Naive Modeling](#fifth-bullet)
* [5. Milestone Modeling + Prophet Model Integration](#sixth-bullet)
* [6. Final Output](#seventh-bullet)

# 0. Raw Data Connections <a class="anchor" id="first-bullet"></a>

In [ ]:
# Read POS Input File
with pyodbc.connect(CONNECTION_STRING, autocommit=True) as conn:
    pos_raw = pd.read_sql("SELECT * FROM default.cbda_pos_model_input", conn)

# Read State Projections
projections = pd.read_csv(f'{BOX_PATH}/Data/Projections/projections_state.csv')
projections_dates = pd.read_csv(f'{BOX_PATH}/Data/Projections/peak_deaths.csv')

# 1. Read in POS Data <a class="anchor" id="second-bullet"></a>

### Filters for POS data

In [ ]:
# Drop prefix created when reading the data from Hive table
pos = pos_raw.copy()
pos.columns = [i.split(".")[1] for i in pos.columns.values]

In [ ]:
pos['week_ending_date'] = pd.to_datetime(pos['week_ending_date'])

# Create a unique identifier for PPG 
cols = ['mdlz_business', 'mdlz_category', 'mdlz_brand','mdlz_ppg']
pos['ppg_id'] = pos[cols].apply(lambda row: '_'.join(row.values.astype(str)), axis=1) 

# Create a unique identifier for PPG/State/Retailer 
cols = ['ppg_id', 'state', 'retailer']
pos['sell_id'] = pos[cols].apply(lambda row: '_'.join(row.values.astype(str)), axis=1) 


# BUSINESS FILTERS PROVIDED BY MDLZ 
pos_main = pos[pos['week_ending_date']>'2019-01-01'] # Week Ending Filter from January 2019
pos_main = pos_main[~pos_main['mdlz_category'].isin(['None','Cookie','Display PRD'])] #Excluding low value categories 
pos_main = pos_main [~((pos_main['mdlz_ppg']=='') | (pos_main['mdlz_ppg'].isnull()))] # Excluding blank and null PPG values
pos_main = pos_main[~((pos_main['mdlz_business']=='') & (pos_main['mdlz_category']=='') & (pos_main['mdlz_brand']=='') & (pos_main['mdlz_ppg']!=''))] # Excluding PPGs with blank product hierarchy
pos_main = pos_main[(pos_main['pos_dollar']>0.0) & (pos_main['pos_qty']>0.0)] # Remove returns
pos_main = pos_main[~(((pos_main['pos_dollar'].isna()) & (pos_main['pos_qty'].isna())))] # Remove null sales


In [ ]:
# Summary Statistics
print ("POS Summary - Before the filters are applied")
print ("\nNumber of rows: {0:,.0f}".format(len(pos)))
print ("Total Dollars: {0:,.0f}".format(np.nansum(pos['pos_dollar'])))
print ("Total Quantity: {0:,.0f}".format(np.nansum(pos['pos_qty'])))
print ("Number of unique products",pos['sell_id'].nunique())

print ("\nPOS Summary - After the filters are applied")
print ("\nNumber of rows: {0:,.0f}".format(len(pos_main)))
print ("Total Dollars: {0:,.0f}".format(np.nansum(pos_main['pos_dollar'])))
print ("Total Quantity: {0:,.0f}".format(np.nansum(pos_main['pos_qty'])))
print ("Number of unique products:",pos_main['sell_id'].nunique())

print ("\n Revenue contribution (%) lost due to the filters",round((1 - (np.nansum(pos_main['pos_dollar'])/np.nansum(pos['pos_dollar'])))*100,2))
print ("\n Volume contribution (%) lost due to the filters",round((1 - (np.nansum(pos_main['pos_qty'])/np.nansum(pos['pos_qty'])))*100,2))

### Calculate the YoY Growth by State, Retailer, PPG (Consider only those weeks which have both 2020 and 2019 sales)

In [ ]:
pos_main['week_of_year'] = pos_main['week_ending_date'].dt.week
pos_main['year'] = pos_main['week_ending_date'].dt.year
pos_main_2020 = pos_main[pos_main['year']==2020]
pos_main_2019 = pos_main[pos_main['year']==2019]
pos_main_2019 = pos_main_2019.drop('week_ending_date', axis = 1)
pos_main_2019 = pos_main_2019.drop(['year','sell_id','ppg_id'], axis = 1)
pos_main_2020 = pos_main_2020.drop('year', axis = 1)
pos_main_new = pos_main_2020.merge(pos_main_2019, on = ['state', 'retailer', 'mdlz_business', 'mdlz_category', 'mdlz_brand', 'mdlz_ppg', 'week_of_year'], how ='left')
pos_main_new = pos_main_new.rename(columns={'pos_qty_x':'pos_qty_ty', 'pos_dollar_x':'pos_dollar_ty', 'pos_qty_y':'pos_qty_ly', 'pos_dollar_y':'pos_dollar_ly'})
pos_main_new['Growth_perc_sales'] = (pos_main_new['pos_dollar_ty'] - pos_main_new['pos_dollar_ly']) / pos_main_new['pos_dollar_ly']
pos_main_new['Growth_perc_qty'] = (pos_main_new['pos_qty_ty'] - pos_main_new['pos_qty_ly']) / pos_main_new['pos_qty_ly']

In [ ]:
latest_pos_week = pos_main_new['week_ending_date'].max()
max_pos_week = pos_main_new['week_of_year'].max()

In [ ]:
print ("Number of unique products after the YoY join",pos_main_new['sell_id'].nunique())

# 2. Read in State Projections <a class="anchor" id="third-bullet"></a>

In [ ]:
def give_week_ending(date):
    start = date - timedelta(days=date.weekday())
    end = start + timedelta(days=5)
    return end

In [ ]:
def give_week_ending_incsun(peak_date):
    dt = datetime.strptime(peak_date, '%Y-%m-%d')
    
    if dt.weekday()!=6:  
        start = dt - timedelta(days=dt.weekday())
        end = start + timedelta(days=5)
    
    if dt.weekday()==6:
        start = dt - timedelta(days=dt.weekday())
        end = start + timedelta(days=12)
    
    return end

In [ ]:
projections_dates['peak_deaths_week'] = projections_dates['peak_deaths_date'].apply(lambda x: give_week_ending_incsun(x))
projections_dates['thirtyperc_deaths_week'] = projections_dates['end_day'].apply(lambda x: give_week_ending_incsun(x))

# 3. Pre-Processing of Modeling Data <a class="anchor" id="fourth-bullet"></a>

### Filtering of Products based on it's 2019 and 2020 Lifespan

In [ ]:
# Reattach the 2019 dataset 
pos_main_2019 = pos_main[pos_main['year']==2019]

# Create a unique identifier for PPG 
cols = ['mdlz_business', 'mdlz_category', 'mdlz_brand','mdlz_ppg']
pos_main_2019['ppg_id'] = pos_main_2019[cols].apply(lambda row: '_'.join(row.values.astype(str)), axis=1) 

# Create a unique identifier for PPG/State/Retailer 
cols = ['ppg_id', 'state', 'retailer']
pos_main_2019['sell_id'] = pos_main_2019[cols].apply(lambda row: '_'.join(row.values.astype(str)), axis=1) 


In [ ]:
# How many product groups have 16 weeks in 2020? 
unique_product_group = pos_main_new.groupby(['sell_id']).agg({'week_ending_date': lambda x: x.nunique()}).reset_index()
print (f"losing number of products with lifespan less than {max_pos_week} weeks of data (%)",round(1 - (len(unique_product_group[unique_product_group['week_ending_date']==max_pos_week]) / len(unique_product_group)),2)*100) # 76% of the product groups 

complete_2020 = unique_product_group[unique_product_group['week_ending_date']==max_pos_week] 
incomplete_2020 = unique_product_group[unique_product_group['week_ending_date']<max_pos_week] 

print (f"Losing 2020 revenue contribution (%) due to dropping products with less than {max_pos_week} weeks of data",round(1-sum(pos_main_new.merge(complete_2020,on=['sell_id'],how='inner')['pos_dollar_ty']) / sum(pos_main_new['pos_dollar_ty']),2)*100)
print (f"Losing 2020 volume contribution (%) due to dropping products with less than {max_pos_week} weeks of data",round(1-sum(pos_main_new.merge(complete_2020,on=['sell_id'],how='inner')['pos_qty_ty']) / sum(pos_main_new['pos_qty_ty']),2)*100)

In [ ]:
# How many product groups have 52 weeks in 2019? 

unique_product_group_2019 = pos_main_2019.groupby(['sell_id']).agg({'week_ending_date': lambda x: x.nunique()}).reset_index()
print ("Losing number of products with lifespan less than 52 weeks of data (%)",round(1 - (len(unique_product_group_2019[unique_product_group_2019['week_ending_date']==52]) / len(unique_product_group_2019)),4)*100) 

complete_2019 = unique_product_group_2019[unique_product_group_2019['week_ending_date']==52] 
incomplete_2019 = unique_product_group_2019[unique_product_group_2019['week_ending_date']<52] 

print ("Losing 2019 revenue contribution (%) due to dropping products with less than 52 weeks of data",round(1-sum(pos_main_2019.merge(complete_2019,on=['sell_id'],how='inner')['pos_dollar']) / sum(pos_main_2019['pos_dollar']),4)*100)
print ("Losing 2019 volume contribution (%) due to dropping products with less than 52 weeks of data",round(1-sum(pos_main_2019.merge(complete_2019,on=['sell_id'],how='inner')['pos_qty']) / sum(pos_main_2019['pos_qty']),4)*100)


In [ ]:
# Products that satisfy the lifespan filter for both 2019 and 2020 - 
print ("2020 products",len(complete_2020))
print ("2019 products",len(complete_2019))
print ("Common products",len(complete_2019.merge(complete_2020,how='inner',on=['sell_id'])))

In [ ]:
common_products = complete_2019.merge(complete_2020,how='inner',on=['sell_id'])
print (f"Losing 2020 & 2019 revenue contribution (%) due to dropping products with less than 52 weeks of 2019 data and {max_pos_week} weeks of 2020 data",round(1-sum(pos_main.merge(common_products,on=['sell_id'],how='inner')['pos_dollar']) / sum(pos_main['pos_dollar']),4)*100)
print (f"Losing 2020 & 2019 volume contribution (%) due to dropping products with less than 52 weeks of 2019 data and {max_pos_week} weeks of 2020 data",round(1-sum(pos_main.merge(common_products,on=['sell_id'],how='inner')['pos_qty']) / sum(pos_main['pos_qty']),4)*100)

### Generate Modeling data for products with insufficient history

In [ ]:
one_df = common_products.merge(incomplete_2020,how='outer',on=['sell_id']) #Adding common_products + incomplete_2020
missing_products = list(set(pos_main_new['sell_id'].unique()) - set(one_df['sell_id'].unique())) # pos_new - one_df  = missing_products sell_id

missing_products_data = pos_main_new[pos_main_new['sell_id'].isin(missing_products)] # Grab missing products data using sell_id
incomplete_2020_data = pos_main_new[pos_main_new['sell_id'].isin(incomplete_2020['sell_id'].unique())] #less than <15 weeks products in 2020

incomplete_data = incomplete_2020_data.append(missing_products_data)

In [ ]:
print ("Check - Number of unique products")
print ("After all the business filters are applied and YoY calculation",pos_main_new['sell_id'].nunique())
print ("Combination of modeling complete data, incomplete 2020 data, missing products",len(common_products) + len(incomplete_2020) + len(missing_products))

# 4. Naive Model - Products with Insufficient History <a class="anchor" id="fifth-bullet"></a>

### Naive Model Declarations

In [ ]:
projections['date'] = pd.to_datetime(projections['date'])
projections['week_of_year'] = projections['date'].dt.week
projections['year'] = projections['date'].dt.year
projections['week_ending_date'] = projections['date'].apply(lambda x: give_week_ending(x))

projections_weekly = projections.groupby(['State','week_ending_date','week_of_year','year']).agg({'deaths_mean':'sum','new_pop_affected':'sum'}).reset_index()
projections_weekly.rename(columns={'State':'state'},inplace=True)

date_list = projections_weekly[projections_weekly['year']==2020].groupby(['week_ending_date','week_of_year']).agg({'state':'count'}).reset_index().sort_values('week_ending_date')

### Model Run

In [ ]:
pos_incomplete = pd.DataFrame()

#product_groups = dataset.groupby(['mdlz_business','mdlz_category','mdlz_brand','mdlz_ppg','retailer','state','ppg_id','sell_id'])
product_groups = incomplete_data.groupby(['mdlz_business','mdlz_category','mdlz_brand','mdlz_ppg','retailer','state','ppg_id','sell_id'])

start_time = time.time()

print(f'Modeling {len(product_groups.groups.keys())} unique sell_ids')
for key in product_groups.groups.keys():
    
    data = product_groups.get_group(key).reset_index()
    
    data = data.merge(date_list[['week_ending_date','week_of_year']],on=['week_ending_date','week_of_year'],how='right').sort_values('week_ending_date')

    data['retailer'] = data["retailer"].fillna(key[4])
    data['state'] = data["state"].fillna(key[5])
    data['mdlz_business'] = data["mdlz_business"].fillna(key[0])
    data['mdlz_category'] = data["mdlz_category"].fillna(key[1])
    data['mdlz_brand'] = data["mdlz_brand"].fillna(key[2])
    data['mdlz_ppg'] = data["mdlz_ppg"].fillna(key[3])
    data['ppg_id'] = data["ppg_id"].fillna(key[6])
    data['sell_id'] = data["sell_id"].fillna(key[7])
    
    # Fill the pos ty with 0 for the actual weeks 
    data.loc[data['week_ending_date']<=latest_pos_week,'pos_dollar_ty'] = data.loc[data['week_ending_date']<=latest_pos_week]['pos_dollar_ty'].fillna(0)
    data.loc[data['week_ending_date']<=latest_pos_week,'pos_qty_ty'] = data.loc[data['week_ending_date']<=latest_pos_week]['pos_qty_ty'].fillna(0)
    
    ros = data.loc[data['week_ending_date']<=latest_pos_week]['pos_qty_ty'].mean()
    
    data['pos_actuals_avg'] = ros
    
    pos_incomplete = pos_incomplete.append(data)

end = time.time()
print("--- %s minutes ---",(end - start_time)/60)

In [ ]:
pos_incomplete.drop(['pos_qty_ly','pos_dollar_ly','index'],axis=1,inplace=True)

pos_incomplete_raw = pos_incomplete.copy()

pos_incomplete = pos_incomplete.merge(pos_main_2019[['sell_id','week_of_year','pos_qty','pos_dollar']],on=['sell_id','week_of_year'],how='left',suffixes=('', '_y'))
pos_incomplete.rename(columns={'pos_qty':'pos_qty_ly','pos_dollar':'pos_dollar_ly'},inplace=True)

In [ ]:
def apply_forecast(week,avg,qty_ty,qty_ly,state):
    
    if week==latest_pos_week:
        return qty_ty
    
    elif week>latest_pos_week and state!='': #short lifespan product forecast
        return avg
    
    elif week>latest_pos_week and state=='': #Null state product forecast
        return qty_ly
    
    else:
        return np.nan

In [ ]:
pos_incomplete['forecast_quantity'] = pos_incomplete[['week_ending_date','pos_actuals_avg','pos_qty_ty',
                                                      'pos_qty_ly','state']].apply(lambda x: apply_forecast(*x),
                                                                                   axis=1)

# 5. Milestone Modeling + Prophet Model - Products with Complete POS History <a class="anchor" id="sixth-bullet"></a>

### Prepare Modeling Data 

In [ ]:
pos_model_data_raw = pos_main_new.merge(common_products['sell_id'],on=['sell_id'],how='inner')
pos_model_data = pos_model_data_raw.copy()
pos_model_data.drop(['retailer','mdlz_business','mdlz_category','mdlz_brand','mdlz_ppg','ppg_id','pos_qty_ly','pos_dollar_ly'],axis=1,inplace=True)

pos_model_data['state_null'] = np.where(pos_model_data.state.isnull(), 1, 0)

In [ ]:
pos_model_data['sell_id'].nunique()

In [ ]:
# Subset the dataset on just the peak covid growth windows (3/14 & 3/21)
maximum_peak_data = pos_model_data[(pos_model_data['week_of_year']>=11) & (pos_model_data['week_of_year']<=12)]

# For each sell id, Identify the growth peak corresponding to those two weeks
sell_maxium_peak = maximum_peak_data.loc[maximum_peak_data.groupby('sell_id')['Growth_perc_qty'].idxmax()][['sell_id','Growth_perc_qty']]
sell_maxium_peak.rename(columns={'Growth_perc_qty':'covid_max_growth'},inplace=True)

pos_model_data = pos_model_data.merge(sell_maxium_peak,on=['sell_id'],how='left')
pos_model_data['negative_covid_impact'] = pos_model_data['covid_max_growth'].apply(lambda x: 1 if x<0 else 0)


In [ ]:
# Calculating 6 week rolling average after sorting the 2019 dataframe
pos_main_2019 = pos_main_2019.sort_values(['sell_id','week_of_year'])
rolling_average_window = 6
pos_main_2019['pos_qty_rolling_6_week_med'] = pos_main_2019.groupby(['sell_id'])['pos_qty'].rolling(window = rolling_average_window).median().reset_index(0,drop=True)


In [ ]:
print ("\nPOS Model data Summary")
print ("\nNumber of rows: {0:,.0f}".format(len(pos_model_data)))
print ("\nNumber of unique products: {0:,.0f}".format(len(pos_model_data['sell_id'].unique())))
print ("Total Dollars: {0:,.0f}".format(np.nansum(pos_model_data['pos_dollar_ty'])))
print ("Total Quantity: {0:,.0f}".format(np.nansum(pos_model_data['pos_qty_ty'])))

print ("\nRevenue contribution (%) lost due to the product filters",round((1 - (np.nansum(pos_model_data['pos_dollar_ty'])/np.nansum(pos_main_2020['pos_dollar']))),3)*100)
print ("\nVolume contribution (%) lost due to the product filters",round((1 - (np.nansum(pos_model_data['pos_qty_ty'])/np.nansum(pos_main_2020['pos_qty']))),3)*100)


### Variable declarations for the model

In [ ]:
# Create a list of future weeks to be projected 
date_list = projections_weekly[(projections_weekly['year']==2020)&(projections_weekly['week_of_year']<=31)].groupby(['week_ending_date','week_of_year']).agg({'state':'count'}).reset_index().sort_values('week_ending_date')
pos_model_data = pos_model_data.sort_values(['sell_id','week_ending_date']).reset_index(drop=True)

### Model Run

In [ ]:
def growth_decay_multiplier(growth):
    
    if growth<0:
        return 0.8
    
    elif growth*100 >= 0 and growth*100 <= 50:
        return 0.2
    
    elif growth*100 > 50 and growth*100 <= 100:
        return 0.15
    
    elif growth*100 > 100 and growth*100 <= 500:
        return 0.1
    
    elif growth*100 > 500 and growth*100 <= 1000:
        return 0.08
    
    else:
        return 0.05    
    
def growth_decay_factor_post_latest_week(milestone):
    
    if milestone==1:
        return 0.9
    
    elif milestone==2:
        return 0.5
    

In [ ]:
pos_merged = pd.DataFrame(columns=['week_ending_date','sell_id','state','pos_qty_ty','pos_dollar_ty',
                                   'week_of_year_x','Growth_perc_sales','Growth_perc_qty','week_of_year_y',
                                   'first_milestone_date','second_milestone_date','third_milestone_date'])

#product_groups = dataset[dataset['state_null']==0].groupby(['sell_id','state'])
product_groups = pos_model_data[pos_model_data['state_null']==0].groupby(['sell_id','state'])

start_time = time.time()
print(f'Modeling {len(product_groups.groups.keys())} unique sell_ids')

for key in product_groups.groups.keys():

    data = product_groups.get_group(key).reset_index() 
    
    state_value = key[1]
    state_null_value = data['state_null'].iat[0]
    covid_impact_value = data['negative_covid_impact'].iat[0]

    
    # To get future weeks for each unique product group
    data = data.merge(date_list[['week_ending_date','week_of_year']], on=['week_ending_date'],how ='outer') 
    
    # Fill in sell_id and state values for the future forecasting weeks 
    data['sell_id'] = data["sell_id"].fillna(key[0])
    data['state'] = data["state"].fillna(key[1])
    data['state_null'] = data["state_null"].fillna(state_null_value)
    data['negative_covid_impact'] = data["negative_covid_impact"].fillna(covid_impact_value)
    
    
    ############################################ Growth Decay Modeling ######################################################
    
    growth_peak_dates = ['2020-03-14','2020-03-21'] 
    death_peak_date = projections_dates[projections_dates['Geography'] == 'United States of America']['peak_deaths_week'].iat[0]
    death_thirty_perc_date = projections_dates[projections_dates['state'] == state_value]['thirtyperc_deaths_week'].iat[0]

    
    # Before covid outbreak median & average growth
    median = data[(data['week_ending_date']>='2020-01-01') & (data['week_ending_date'] <'2020-03-01')]['Growth_perc_qty'].median() 
    average = data[(data['week_ending_date']>='2020-01-01') & (data['week_ending_date'] <'2020-03-01')]['Growth_perc_qty'].mean()
    
    # Find the absolute peak growth value and it's corresponding week 
    peak_growth_value = data.loc[data[(data['week_ending_date']>=growth_peak_dates[0]) & (data['week_ending_date']<=growth_peak_dates[1])]['Growth_perc_qty'].idxmax()]['Growth_perc_qty']
    peak_growth_date = data.loc[data[(data['week_ending_date']>=growth_peak_dates[0]) & (data['week_ending_date']<=growth_peak_dates[1])]['Growth_perc_qty'].idxmax()]['week_ending_date']
    
    
    data['forecast1'] = 0.0
    data['forecast_quantity'] = 0.0

    
    # Replicate the peak growth value as the very first forecast value - just for visualization (including stop gap measure)
    latest_growth_value = data.loc[data['week_ending_date'] == latest_pos_week]['Growth_perc_qty'].iat[0]
    latest_pos_value = data.loc[data['week_ending_date'] == latest_pos_week]['pos_qty_ty'].iat[0]

    
    data.loc[(data['week_ending_date'] == latest_pos_week),'forecast1'] = latest_growth_value
    data.loc[(data['week_ending_date'] == latest_pos_week),'forecast_quantity'] = latest_pos_value

    ############################################  Milestone Modeling     ###################################
    
    #     POS under social distancing: latest pos week - second milestone date 
    #     ramp down: second milestone date - third milestone date =  2 weeks window
    #     new normal: >third milestone date 
    
    
    first_milestone_date = death_peak_date #US specific
    second_milestone_date = death_thirty_perc_date
    
      
    # Shift the second milestone date only
    if (second_milestone_date - latest_pos_week).days/7 >=6:
        second_milestone_date = death_thirty_perc_date - timedelta(days=28)
        
    state_open_date = second_milestone_date + timedelta(days=14) 
    
    
    ############### Check Forecast Type  ###############
    
    if (second_milestone_date - latest_pos_week).days/7 <=2:
        data['fc_type'] = 1
        
        # New Normal Forecast
        data.loc[(data['week_ending_date']> latest_pos_week),'forecast1'] = median
    
    elif (second_milestone_date - latest_pos_week).days/7 ==3:
        data['fc_type'] = 2
        
        # Ramp Down period
        second_milestone_range = data.loc[(data['week_ending_date'] >= latest_pos_week) & (data['week_ending_date']< state_open_date),'forecast1'].index.values

        
        for i in range(second_milestone_range[0], second_milestone_range[-1]+1):
            data.loc[i+1, 'forecast1'] = data.loc[i, 'forecast1'] * growth_decay_factor_post_latest_week(2)
   
        # New Normal period
        data.loc[(data['week_ending_date']>= state_open_date),'forecast1'] = median
    
    
    elif (second_milestone_date - latest_pos_week).days/7 > 3:
        data['fc_type'] = 3
    
        # POS Under social distancing 
        first_milestone_range = data.loc[(data['week_ending_date'] >= latest_pos_week) & (data['week_ending_date'] < second_milestone_date),'forecast1'].index.values


        for i in range(first_milestone_range[0], first_milestone_range[-1]+1):
            data.loc[i+1, 'forecast1'] = data.loc[i, 'forecast1'] * growth_decay_factor_post_latest_week(1)

            if data.loc[i,'negative_covid_impact']==0:
                data.loc[i+1, 'forecast_quantity'] = data.loc[i, 'forecast_quantity'] * growth_decay_factor_post_latest_week(1)

            else:
                data.loc[i+1, 'forecast_quantity'] = data.loc[i, 'forecast_quantity'] / growth_decay_factor_post_latest_week(1)

        # Ramp Down 
        second_milestone_range = data.loc[(data['week_ending_date'] >= second_milestone_date) & (data['week_ending_date']< state_open_date),'forecast1'].index.values

        data.loc[second_milestone_range[0]:second_milestone_range[0]+1, 'forecast_quantity'] = 0   # Setting this as 0 in the first milestone range = 1 week only

        for i in range(second_milestone_range[0], second_milestone_range[-1]+1):
            data.loc[i+1, 'forecast1'] = data.loc[i, 'forecast1'] * growth_decay_factor_post_latest_week(2)

        # New Normal 
        data.loc[(data['week_ending_date']>= state_open_date),'forecast1'] = median

        
    data['first_milestone_date'] = death_peak_date 
    data['second_milestone_date'] = second_milestone_date
    data['third_milestone_date'] = state_open_date
    data['median_baseline'] = median
    
    pos_merged = pos_merged.append(data)

end = time.time()
print("--- %s minutes ---",(end - start_time)/60)

In [ ]:
# Save a copy of the model output df 
pos_merged_raw = pos_merged.copy()

### Post-Processing of the Milestone Model Output & Integration of Prophet Model Output

In [ ]:
pos_merged = pos_merged.drop(['week_of_year_x','index','covid_max_growth'],axis=1)
pos_merged.rename(columns={'week_of_year_y':'week_of_year'},inplace=True)


In [ ]:
# Merge model output with 2019 dataset to obtain the pos_qty_ly
pos_merged = pos_merged.merge(pos_main_2019,on=['sell_id','week_of_year'],how='left',suffixes=('', '_y'))
pos_merged.rename(columns={'pos_qty':'pos_qty_ly','pos_dollar':'pos_dollar_ly'},inplace=True)
pos_merged.drop(['week_ending_date_y','state_y','year'],axis=1,inplace=True)


In [ ]:
# Read in Prohpet's forecasts
prht_fcst_common = pd.read_feather(f'{BOX_PATH}/Model Output/POS Prophet Model/pos_0430_cleaned_v2.feather')

# Calculate rolling 6 week median of prophet's forecast to exclude effect of promotions
prht_fcst_common = prht_fcst_common.sort_values(['sell_id','week_ending_date'])
rolling_average_window = 6
prht_fcst_common['pos_qty_prht_rolling_6_week_med'] = prht_fcst_common.groupby(['sell_id'])['prht_frcst'].rolling(window = rolling_average_window).median().reset_index(0,drop=True)

prht_fcst_common = prht_fcst_common.drop(['prht_frcst'], axis=1)

# prht_fcst_common.head(3)

In [ ]:
# Merge in prophet's forecast for common products
pos_merged = pos_merged.merge(prht_fcst_common, how='left', on=['sell_id','week_ending_date'])
#pos_merged = pos_merged.merge(prht_fcst_high_grwth, how='left', on=['sell_id'])

# Use prophet's forecast if available
pos_merged = pos_merged.assign(pos_qty_rolling_6_week_med = \
                              np.where(~(pos_merged.pos_qty_prht_rolling_6_week_med.isnull()), 
                                       pos_merged.pos_qty_prht_rolling_6_week_med, 
                                       pos_merged.pos_qty_rolling_6_week_med))

pos_merged = pos_merged.drop(['pos_qty_prht_rolling_6_week_med'],axis=1)

# pos_merged.head(3)

In [ ]:
# Subset the dataset on just the peak covid growth windows (3/14 & 3/21)
maximum_peak_data = pos_merged[(pos_merged['week_of_year']>=11) & (pos_merged['week_of_year']<=12)]

# For each sell id, Identify the growth peak corresponding to those two weeks
sell_maxium_ly = maximum_peak_data.loc[maximum_peak_data.groupby('sell_id')['Growth_perc_qty'].idxmax()][['sell_id','pos_qty_ly']]
sell_maxium_ly.rename(columns={'pos_qty_ly':'max_pos_ly'},inplace=True)

pos_merged = pos_merged.merge(sell_maxium_ly,on=['sell_id'],how='left')

In [ ]:
# Adjust forecast quantity for positive growing products by controlling for promotions
def apply_forecast_value_positive(week,growth,qty_ly_rolling,qty_ty,qty_ly_max,z,third_m,second_m,fcst_qty,high_growth_decline):
      
    if week == latest_pos_week: # If the week is current pos week, set the forecast value to actual pos (just for viz purporse)
        return qty_ty
    
    elif week>latest_pos_week and week<second_m:
        return fcst_qty
    
    elif week>=second_m and high_growth_decline==True:
        return qty_ly_rolling
    
    elif week>=second_m:
        return (1 + growth) * qty_ly_rolling   
    
    else:
        return np.nan

In [ ]:
# Adjust forecast quantity for negative growing products by controlling for promotions
def apply_forecast_value_negative(week,growth,qty_ly_rolling,qty_ty,qty_ly_max,z,third_m,second_m,fcst_qty,high_growth_decline):
      
    if week == latest_pos_week: # If the week is current pos week, set the forecast value to actual pos (just for viz purporse)
        return qty_ty
    
    elif week>latest_pos_week and week<second_m:
        return fcst_qty
    
    elif week>=second_m and week<third_m:
        return (1 + growth) * qty_ly_max    
    
    elif week>=third_m and high_growth_decline==True:
        return qty_ly_rolling
    
    elif week>=third_m:
        return (1+growth) * qty_ly_rolling
    
    else:
        return np.nan

In [ ]:
# Adjust forecast growth for positive growing products by controlling for outliers
def floor_growth(median,growth,week,z): 
        
    if week>latest_pos_week and growth<median:
        return median
    else:
        return growth
    

In [ ]:
# Adjust forecast growth for negative growing products by controlling for outliers
def ceil_growth(median,growth,week,z): 
    
    if week>latest_pos_week and growth>median:
        return median
    else:
        return growth
    

In [ ]:
pos_merged['forecast1'] = pos_merged[['median_baseline','forecast1','week_ending_date','negative_covid_impact']].\
                            apply(lambda x: floor_growth(*x) if x[3] == 0 else ceil_growth(*x),axis=1)

pos_merged['forecast_quantity'] = pos_merged[['week_ending_date','forecast1','pos_qty_rolling_6_week_med',
                                              'pos_qty_ty','max_pos_ly','negative_covid_impact',
                                              'third_milestone_date','second_milestone_date','forecast_quantity',
                                              'high_growth_decline']]\
                                    .apply(lambda x: apply_forecast_value_positive(*x) if x[5]==0  \
                                            else apply_forecast_value_negative(*x) ,axis=1)


In [ ]:
# Update final forecast quantity by controlling for outliers
def update_forecast_positive(week,forecast1,fcst_growth,pos_qty_rolling,fcst_qty,first_m,third_m,neg_covid, high_growth_decline):

    if week>latest_pos_week and week<third_m and high_growth_decline!=True: # Floor the value 
        if fcst_growth < forecast1:
            return (1+forecast1) * pos_qty_rolling
        else:
            return fcst_qty
    
    else:
        return fcst_qty

In [ ]:
# Update final forecast quantity by controlling for outliers
def update_forecast_negative(week,forecast1,fcst_growth,pos_qty_rolling,fcst_qty,first_m,third_m,neg_covid, high_growth_decline):

    if week>latest_pos_week and week<third_m and high_growth_decline!=True: # Ceil the value
        if fcst_growth > forecast1:
            return (1+forecast1) * pos_qty_rolling
        else:
            return fcst_qty
    
    else:
        return fcst_qty

In [ ]:
def find_promo_uplift(week,second_m,qty_ly,qty_rolling):
    
    if week>=second_m:
        return qty_ly - qty_rolling
    else:
        return 0
    

In [ ]:
pos_merged['forecast_growth'] = (pos_merged['forecast_quantity'] - pos_merged['pos_qty_ly']) / (pos_merged['pos_qty_ly'])

pos_merged['forecast_quantity_new'] = pos_merged[['week_ending_date','forecast1','forecast_growth',
                                                  'pos_qty_rolling_6_week_med','forecast_quantity',
                                                  'first_milestone_date','third_milestone_date',
                                                  'negative_covid_impact','high_growth_decline']]\
                                            .apply(lambda x: update_forecast_positive(*x) if x[7]==0 else 
                                                   update_forecast_negative(*x) ,axis=1)

pos_merged['promo_uplift'] = pos_merged[['week_ending_date','second_milestone_date','pos_qty_ly','pos_qty_rolling_6_week_med']].apply(lambda x: find_promo_uplift(*x),axis=1)

pos_merged.drop(['forecast_quantity'],axis=1,inplace=True)
pos_merged.rename(columns={'forecast_quantity_new':'forecast_quantity'},inplace=True)

pos_merged.loc[pos_merged['week_ending_date']<latest_pos_week,'forecast1'] = np.nan
pos_merged.loc[pos_merged['week_ending_date']<latest_pos_week,'forecast_quantity'] = np.nan
pos_merged.loc[pos_merged['week_ending_date']<latest_pos_week,'forecast_growth'] = np.nan
pos_merged.loc[pos_merged['week_ending_date']<latest_pos_week,'promo_uplift'] = np.nan

# Save a copy for backup! 4/27 at 11:07 pm 
pos_merged_copy = pos_merged.copy()

In [ ]:
pos_merged = pos_merged.sort_values(['sell_id','week_ending_date'])

In [ ]:
print ("Without merging with the states data, number of rows", len(pos_merged))
print ("Without merging with the states data, number of product groups", int(len(pos_merged)/32))

# 6. Final Output <a class="anchor" id="seventh-bullet"></a>

### Merging Naive, Milestone Model and Prophet Model outputs

In [ ]:
# Merging modeling and short lifespan datasets
pos_merged1 = pos_merged.append(pos_incomplete)

In [ ]:
pos_merged1['sell_id'].nunique()

In [ ]:
pos_merged1['week_ending_date'] = pd.to_datetime(pos_merged1['week_ending_date'])
pos_merged1['first_milestone_date'] = pd.to_datetime(pos_merged1['first_milestone_date'])
pos_merged1['second_milestone_date'] = pd.to_datetime(pos_merged1['second_milestone_date'])
pos_merged1['third_milestone_date'] = pd.to_datetime(pos_merged1['third_milestone_date'])


In [ ]:
print ("After merging with the states data, number of rows", len(pos_merged1))
print ("After merging with the states data, number of product groups", int(len(pos_merged1)/32))


In [ ]:
# Write out Model output parquet file to edge node
start = time.time()

out_buffer = BytesIO()
pos_merged1.to_parquet(out_buffer, index=False)

destination_path = "pos_model_results.parquet.gzip"
with edge_node.open(destination_path, 'w+', 32768) as f:
    f.write(out_buffer.getvalue())
    
print('Writing out took', time.time()-start, 'seconds.')